# Covid Brasil

In [ ]:
#%config IPCompleter.greedy=True
#!pip install pandas
#!pip install seaborn
#!pip install plotly
#!pip install altair

In [53]:
import pandas as pd 
import numpy as np 
import seaborn as sns

from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt 
import ipywidgets as widgets
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import altair as alt
print('Modules are imported.')

Modules are imported.


In [54]:
import io
import requests
url='https://data.brasil.io/dataset/covid19/caso.csv.gz'
s=requests.get(url).content


In [55]:
with open('data/covidio.csv.gz', 'wb') as f:
    f.write(s)

In [56]:
#df = pd.read_csv('data/covidio.csv.gz', parse_dates=['last_available_date'], index_col='last_available_date')
df = pd.read_csv('data/covidio.csv.gz')


In [57]:
df.head()

,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,estimated_population_2019,estimated_population,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,2021-03-03,AP,NaN,state,84483,1146,345,True,845731.0,861773.0,16.0,9803.39370,0.0136
1,2021-03-02,AP,NaN,state,84177,1144,344,False,845731.0,861773.0,16.0,9767.88551,0.0136
2,2021-03-01,AP,NaN,state,83885,1142,343,False,845731.0,861773.0,16.0,9734.00188,0.0136
3,2021-02-28,AP,NaN,state,83663,1140,342,False,845731.0,861773.0,16.0,9708.24103,0.0136
4,2021-02-27,AP,NaN,state,83505,1139,341,False,845731.0,861773.0,16.0,9689.90674,0.0136


In [58]:
df['mortalidade'] =  df['deaths']/df['estimated_population']
df['mortalidade100k'] = (df['deaths']*100000)/df['estimated_population']
df[['mortalidade', 'death_rate', 'mortalidade100k']].head()

,mortalidade,death_rate,mortalidade100k
0,0.001330,0.0136,132.981655
1,0.001327,0.0136,132.749576
2,0.001325,0.0136,132.517496
3,0.001323,0.0136,132.285416
4,0.001322,0.0136,132.169376


In [60]:
df.drop(df[df['deaths'] == 0].index, inplace=True)

In [61]:
df_states = df[(df['is_last'] == True) &(df.place_type=='state')][['state','confirmed','deaths', 'death_rate','mortalidade','mortalidade100k']]
df_cities = df[(df['is_last'] == True) &(df.place_type=='city')][['city','state', 'confirmed','deaths', 'death_rate','mortalidade','mortalidade100k']]

In [62]:
# deletando linhas inuteis
df_cities.drop(df_cities[df_cities['city'] == 'Importados/Indefinidos'].index, inplace=True)

In [63]:
df_states.columns = ['Estado', 'Confirmados', 'Mortes', 'Letalidade','Mortalidade', 'Mortalidade100k']
df_cities.columns = ['Cidade', 'Estado', 'Confirmados', 'Mortes', 'Letalidade','Mortalidade', 'Mortalidade100k']

In [ ]:
#df_cities.sort_values('Letalidade',ascending=False).head()
df_states['Letalidade_rank'] = df_states['Letalidade'].sort_values(column,ascending=asc).rank(method='min').astype(int)
df_states['Mortalidade_rank'] = df_states['Mortalidade'].rank(method='min').astype(int)
df_states['rank'] = ((df_states['Mortalidade_rank'] + df_states['Letalidade_rank'])/2).rank(method='min').astype(int)
df_cities['Letalidade_rank'] = df_cities['Letalidade'].rank(method='min').astype(int)
df_cities['Mortalidade_rank'] = df_cities['Mortalidade'].rank(method='min').astype(int)
df_cities['rank'] = ((df_cities['Mortalidade_rank'] + df_cities['Letalidade_rank'])/2).rank(method='min').astype(int)

In [ ]:
def search_city(city, color='red', reverse=False):
    return df_cities[df['city'] == city]

def style_frame(df):
    cm = sns.light_palette(color, as_cmap=True, reverse=reverse)
    return (df.style.set_properties(**{'text-align': 'right'}).background_gradient(cmap=cm).hide_index()).set_caption(
    title)

def treemap_show(df=df_states, nivel='Estado', column='rank', n=10,  asc=False, title=""):
    fig = px.treemap(df.head(30), 
                 path=[nivel], 
                 values=column, height=700,
                 title=title,
                 color_discrete_sequence = px.colors.qualitative.Prism)

    fig.data[0].textinfo = 'label+text+value'
    #fig.write_image("images/fig2.png")

    fig.show()


def covid_data_table(df, nivel, column, n, asc=False,title="", color='red', reverse=False, corte=0):

    if(corte > 0):
        df = df[df['Mortes'] > corte]
    
    df = df.sort_values(column,ascending=asc).copy().head(n)

    data_table = pd.DataFrame()
    data_table[nivel] = df[nivel]
    if nivel == 'Cidade':
        data_table['Estado'] = df['Estado']
    #data_table["Confirmados"] = df['Confirmados']
    data_table["rank"] = df['rank']
    data_table["Letalidade_rank"] = df['Letalidade_rank']
    data_table["Mortalidade_rank"] = df['Mortalidade_rank']
    data_table["Mortes"] = df['Mortes']
    data_table["Mortalidade100k"] = df['Mortalidade100k'].astype(int)
    data_table["Letalidade"] = df['Letalidade']
    
    #data_table["Mortalidade"] = df['Mortalidade']
    
    cm = sns.light_palette(color, as_cmap=True, reverse=reverse)
    return (data_table.style.set_properties(**{'text-align': 'right'}).background_gradient(cmap=cm).hide_index()).set_caption(
    title)

def tabela_estado(column, n, asc=False,title="", color='red', reverse=False, corte=0):
    '''Retorna uma table para estado '''
    return covid_data_table(df_states,'Estado', column, n, asc=asc,title=title, color=color, reverse=reverse, corte=corte)

def tabela_estado_reversa(column, n, title="", corte=0):
    return tabela_estado(column, n, True, title, 'green', True, corte)

def tabela_cidade(column, n, asc=False,title="", color='red', reverse=False, corte=0):
    '''Retorna uma table para cidades '''
    return covid_data_table(df_cities,'Cidade', column, n, asc=asc,title=title, color=color, reverse=reverse, corte=corte)

def tabela_cidade_reversa(column, n, title="", corte=0):
    return tabela_cidade(column, n, True, title, 'green', True, corte=corte)

## Top 10 Piores no Ranking (Letalidade + Mortalidade)/2

In [ ]:
tabela_estado('rank', 10)

## Top 10 Melhores Estados no Ranking

In [ ]:
tabela_estado_reversa('rank',10)

## Top  20 piores cidades do brasil com +500 Obitos

In [ ]:
tabela_cidade('rank',20, corte=500)

## Top 20 Cidades do Brasil

In [ ]:
tabela_cidade_reversa('Mortalidade',20,corte=1)

## Top 10 menor letalidade 

In [ ]:
def covid_bubble_chart(df, column, n, asc=False):
    df = df.sort_values(column,ascending=asc).copy().head(n)
    fig = px.scatter(df, x="Estado", y=column, size=column, color="Estado",
               text="Estado", size_max=90)
    fig.update_layout(
    title="Os top " + str(n) + ' piores estados por '  + column,
    xaxis_title="Estados",
    yaxis_title=column,
    width = 700
    )
    fig.show();
    
covid_bubble_chart(df_states, 'Letalidade',10, True)


In [ ]:
covid_bubble_chart(df_states, 'Mortes',10)

In [ ]:
search_city("Campanha")